In [8]:
# THIS notebook is based on C:\SCRIPTS/RF_imageClass_MPB_p27_FT_3.py
#environment space is located in "img_proc2"
#November 2020
#NASA ARSET: Land Cover Classification with Radar and Optical Data, Part 2/4
#https://www.youtube.com/watch?v=raXA3gnb94Q  is RF on Google Earth

### 1. import libraries

In [2]:
# import os,  numpy as np
# from gdalconst import *
# import pandas as pd
# # import gdal,sys, time
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier
# #for visualization
# import seaborn as sn
# import matplotlib.pyplot as plt
# # Set random seed. if it is not set up then every run would produce different set of important bands
# np.random.seed(0)
# %matplotlib inline
# A list of "random" colors (for a nicer output)
COLORS = ["#000000", "#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941"]

In [3]:
#libraries for accuracy assessment

from sklearn.metrics import  accuracy_score, f1_score,confusion_matrix, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import auc
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

### 2. Create some functions
to process vector files, write geotiffs and evaluate models

In [5]:
def create_mask_from_vector(vector_data_path, cols, rows, geo_transform, projection, target_value=1):
    """Rasterize the given vector (wrapper for gdal.RasterizeLayer)."""
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)                                #???????????????????????
    featureCount = layer.GetFeatureCount()
    print("Number of features in %s: %d" % (vector_data_path, featureCount))
    driver = gdal.GetDriverByName('MEM')  # In memory dataset
    #target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds = driver.Create('', cols, rows, 1,gdal.GDT_Float32)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds


def vectors_to_raster(file_paths, rows, cols, geo_transform, projection):
    """Rasterize all the vectors in the given directory into a single image. all
    vectors get value of order they are in ."""
    labeled_pixels = np.zeros((rows, cols))
    for i, path in enumerate(file_paths):
        
        label = i+1
        #print("label:", label, "path:", path)
        ds = create_mask_from_vector(path, cols, rows, geo_transform, projection, target_value=label)
        band = ds.GetRasterBand(1)
        labeled_pixels += band.ReadAsArray()
        ds = None
        #print("lable pixel:", labeled_pixels)
    return labeled_pixels


def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

def evaluate(model, test_features, test_labels):
    "evaluate accuracy of the model"
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

    return accuracy

### 3. set up paths and open the image

In [6]:

os.chdir(r"C:/_LOCALdata/_PROJECTS_hist/ASLC/2020/F20")
print("wdir is:", os.getcwd())
# gdalbuildvrt -separate RGB.vrt red.tif green.tif blue.tif
# gdal_translate RGB.vrt RGB.tif
#raster_data_path = r"D:\...\image_\LC08_048020_20190616_S1_slpTPITRI_sub_1.tif"
#raster_data_path = r"C:/....0/image/F_20_stack_virt"
#stack with Alos2 intensity
#raster_data_path = r"C:/_L....020/F20/image/F_20_stack_Alos2_int_virt"
#raster_data_path = r"C:\_LO......age\Optic\a_Sent_2_4_alos_26911_20m.tif"
raster_data_path = r"C:/_L.....mage/F_20_stack_new1.tif"
t_fname = "clas_pilot.tif"
red_output = "clas_red_pilot.tif"
#train folder has got shapefiles with vector data to be used for training
train_data_path = r"C:/_L...../vectors/Alos_train_1"

#test folder  is similar to the train directory, but this samples are going to be used to
#verify the classification results.
validation_data_path = r"C:/_.....ctors/Alos_test_1"

#raster_dataset = gdal.Open(raster_data_path, GA_ReadOnly)
raster_dataset = gdal.Open(raster_data_path)
if raster_dataset is None:
    print ('Unable to open INPUT.tif')
    sys.exit(1)
cols = raster_dataset.RasterXSize
#raster_dataset = gdal.Open(raster_data_path, gdal.GA_ReadOnly)
geo_transform = raster_dataset.GetGeoTransform()
proj = raster_dataset.GetProjectionRef()
numbOfBands = raster_dataset.RasterCount
print("#ofbands:" , numbOfBands)
print("cols:",cols)
print("gtr:",geo_transform)
print("projection:", proj)
bands_data = []
#exclude 1st L8 "Coastal band"
#for b in range( raster_dataset.RasterCount):  #exclude 1st band
#read data into a stack.....
for b in range(    numbOfBands):
    print("b_:", b, "=", b+1)

    band = raster_dataset.GetRasterBand(b+1)
    bands_data.append(band.ReadAsArray())

bands_data = np.dstack(bands_data)
rows, cols, n_bands = bands_data.shape
print("row,col,band:",rows, cols, n_bands)

### 4.a process training data 

In [ ]:

files = [f for f in os.listdir(train_data_path) if f.endswith('.shp')]
classes = [f.split('.')[0] for f in files]
shapefiles = [os.path.join(train_data_path, f)  for f in files if f.endswith('.shp')]
print("shp:", shapefiles)

#create classes
labeled_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform, proj)
is_train = np.nonzero(labeled_pixels)
#show row and col index for each label pixel
print("is_train:", is_train)
training_labels = labeled_pixels[is_train]   #show the value of the label pixel or agcc class
#print("tr_labe:",training_labels)

#show value of the  pixels in each band under training polygon
training_samples = bands_data[is_train]
#print("tr_samp:",training_samples )
# print("tl:", training_labels.shape)
# print("ts:",training_samples.shape)

training_labels
clase = np.unique(training_labels)
print("trainig includes {n}  classes with codes: {classes}  " .format (n= clase.size, classes= clase))


### 4.b ...and process validation data 

In [ ]:
#***************************************process validation (Test) files
# they will be used to Assess the results
 #put valid data into a list
    
v_files = [fv for fv in os.listdir(validation_data_path) if fv.endswith('.shp')] 
#v_shapefiles = [os.path.join(validation_data_path, "%s.shp"%c) for c in classes]
v_classes = [fv.split('.')[0] for fv in v_files]
v_shapefiles = [os.path.join(validation_data_path, fv) for fv in v_files if fv.endswith('.shp')]
print(v_shapefiles)

verification_pixels = vectors_to_raster(v_shapefiles, rows, cols, geo_transform, proj)
is_test = np.nonzero(verification_pixels) #2 arrays one for cols and another for rows pointing test samples
testing_labels = verification_pixels[is_test]   #show the value of the label pixel or agcc class
print("test_labe:",testing_labels)
#pixel values for each important band undet test sample
#TESTING data -- pixel values oftesting samples......
testing_samples = bands_data[is_test]

### 5. Initiate model

In [ ]:
classifier = RandomForestClassifier(random_state=42)
#PARAMS
#********************  train the bse model   ****************
classifier.fit(training_samples, training_labels)  #

##Make Predictions on Test Data for base model

predicted_labels_0 = classifier.predict(testing_samples )
## Determine Performance Metrics
matrix = confusion_matrix(testing_labels, predicted_labels_0)
print('Confusion matrix : \n')
print( matrix)
# outcome values order in sklearn

# classification report for precision, recall f1-score and accuracy
matrix_rep = classification_report(testing_labels, predicted_labels_0)
print('Classification report : ' )
print(matrix_rep)

print('initial_Correct red Prediction (%): ', accuracy_score(testing_labels,predicted_labels_0, normalize=True)*100.0)
#

### 6. FEATURE IMPORTANCES
 finds which raster input bands are important. those corelated with others are removed

In [ ]:
##################   Variable Importances
#our model is trained now on training data! MOdel can be trained on entire data as well
######################################################
print("#ofbands:" , n_bands)
listOfBands = []
for i  in range(1, numbOfBands +1):
    print("band:", i)
   # listOfBands.append(i+1)
    listOfBands.append(i )
print("lofB:", listOfBands)

for b, imp in zip(listOfBands,classifier.feature_importances_):
    print("band: {b} importance: {imp}" .format(b=b, imp=imp))

#What bands are the most important for clasified output
# Get numerical feature importances
importances = list(classifier.feature_importances_  ) #put in list feature_importances_
#
# # List of tuples with variable and importance

feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(listOfBands, importances)]

# # Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# # Print out the feature and importances
for pair in feature_importances:
    print('Variable: {:10} Importance: {}'.format(*pair))
#[print('Variable: {:10} Importance: {}'.format(*pair)) for pair in feature_importances];

# #**************** Create a plot of most important bands**********************************
# #***********************************************************************************
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(1,len(importances)+1))
print("xval:",x_values)
# # Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)

# # Tick labels for x axis
plt.xticks(x_values, listOfBands, rotation='horizontal')

# # Axis labels and title
plt.ylabel('Importance'); plt.xlabel('bands'); plt.title('Variable Importances');
plt.show()
plt.savefig('bars_S2.png')

### 7. SORTED IMPORTANCES 
now we sort them according to importance. the least important are thrown away

In [ ]:
# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
print("sorted importances:",sorted_importances)
print("sorted bands:",sorted_features)  #
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
print("ci:",cumulative_importances )
# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')
print(x_values)
# # Draw line at 95% of importance retained
plt.hlines(y = 0.95, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')

# # Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')

# # Axis labels and title
plt.xlabel('bands'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');
plt.show()
plt.savefig('cumusumm_S2.png')

### 8. DATA REDUCTION
now we have created new dataset with less bands

In [ ]:
# Find number of features for cumulative importance of 95%
# Add 1 because Python is zero-indexed
needed_bands = np.where(cumulative_importances > 0.95)[0][0] + 1
print('Number of features for 95% importance:', np.where(cumulative_importances > 0.95)[0][0] + 1)
print("band:",raster_dataset.RasterCount)
print("nb:",needed_bands)
if (needed_bands < raster_dataset.RasterCount  ):
    print(" need for data reduction          ")
else:
    print(" no data reduction necessary")
# # Extract the names of the most important features
# CHANGE NUMBER OF FEATURES FOR 95 IMPORTANCES [0:????]
important_feature_names = [feature[0] for feature in feature_importances[0:needed_bands]]
# Find the columns of the most important features
important_indices = [listOfBands.index(feature) + 1 for feature in listOfBands]
# feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(bands, importances)]
#print("imp:", important_indices)
print("imp name:", important_feature_names)
alist = []
for a in range(needed_bands):
   # print("aband: ",important_feature_names [a])
    alist.append(important_feature_names [a])
alist = sorted(alist)

imp_bands_data = []
#for bs in important_feature_names:  # exclude 1st band
for bs in alist:
    iband = raster_dataset.GetRasterBand(bs)
    ar = iband.ReadAsArray()
    imp_bands_data.append(ar)

#new stack with important bands is "imp_bands_data"
imp_bands_data = np.dstack(imp_bands_data)
row, col, imp_bands = imp_bands_data.shape
print("imp_shp:", imp_bands_data.shape)

### 9. Training the RF model on reducted data set

In [ ]:

#
#show row and col index for each label pixel #show value of the  pixels in each band under training polygon
#*********RUN CLASIFIER ON TRAINING DATA WITH REDUCTED DATASET***********************
#LABELS  are the same , but imp_traing pixels are different due to less number of bands

print("train_lab_shp:",training_labels.shape)
imp_training_samples = imp_bands_data[is_train]
imp_testing_samples = imp_bands_data[is_test]
print("test & train_sampl_shp:",imp_training_samples.shape, imp_testing_samples.shape)
# #***********************
###        Model with The Most Important Features***************
#START model on feature enginered*******************************************************************************************************************************

classifier_imp = RandomForestClassifier(criterion = 'entropy',max_features= 'sqrt', n_jobs=-1,n_estimators=100,oob_score=True)
classifier_imp.fit(imp_training_samples, training_labels)
predicted_imp_labels = classifier_imp.predict(imp_testing_samples )
print('initial_Correct import Prediction (%): ', accuracy_score(testing_labels,predicted_imp_labels, normalize=True)*100.0)
accur_4_imp = evaluate(classifier_imp,imp_testing_samples,testing_labels)
accur_4_base = evaluate(classifier, training_samples, training_labels)

In [ ]:
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters for classifier currently in use:\n')
pprint(classifier.get_params())
print('Parameters for IMP classifier currently in use:\n')
pprint(classifier_imp.get_params())

###  10. FINE TUNING of the model
we try to find best parameters for RF model using
**Random Search with Cross Validation**  producing highest accuracy

In [ ]:

print( "Crosswalidation...Use the random grid to search for best hyperparameters.")
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print("these is random_grid parameters")
pprint(random_grid)
# First create the base model to tune
tune_classifier_rf = RandomForestClassifier(random_state=42)

# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=tune_classifier_rf, param_distributions=random_grid,
                              n_iter = 100, scoring='neg_mean_absolute_error',
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,  return_train_score=True)

# Fit the random search model
rf_random.fit(imp_training_samples, training_labels);
print("these are the best params random search....")
print( rf_random.best_params_)
#print long list of results
#rf_random.cv_results_

In [ ]:
### Evaluate the Default Model

best_random = rf_random.best_estimator_
print("after grid search evaluation of th best random model: ")
random_accuracy = evaluate(best_random, imp_testing_samples, testing_labels)
print("rand_acuur: ",random_accuracy)

### 11. GRID SEARCH 
Now, we do grid search based on best parameters from RandomizedSearch. 
We test a range of hyperparameters around best values returned from randomsearch.

In [ ]:

####################################*********2*****************************
# Create the parameter grid based on the results of random search
param_grid = {
    'bootstrap': [True],
    'max_depth': [ 90, 100, 110, 120],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [70, 80, 90, 110]
}
# Create a base model
rf_gs =RandomForestClassifier(random_state = 42)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf_gs, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)
grid_search.fit(imp_training_samples, training_labels);

print("grid_search.best_params_:",grid_search.best_params_)
#Evaluate the Best Model from Grid Search

best_grid = grid_search.best_estimator_
gs_accuracy = evaluate(best_grid , imp_testing_samples, testing_labels)
print("gs_acuur: ",gs_accuracy)

### 12. AUC_ROC curve for multiclass classification
it uses OneVsRestClassifier tehnique
plots true positive rates vs false positive rates


In [ ]:
classifier_imp2= OneVsRestClassifier(RandomForestClassifier(bootstrap= True, min_samples_leaf= 3,n_estimators= 90, min_samples_split= 8, max_features = 2, max_depth= 90))

#classifier_imp2=clf = OneVsRestClassifier(RandomForestClassifier())#for ROC curve
classifier_imp2.fit(imp_training_samples, training_labels)  #
#classifier_imp2= OneVsRestClassifier(best_grid)
predicted_labels_roc = classifier_imp2.predict(imp_testing_samples )
pred_prob_labels_roc = classifier_imp2.predict_proba(imp_testing_samples )

fpr = {}
tpr = {}
thresh ={}
n_class = len(v_files)
for i in range(len(v_files)):
    fpr[i], tpr[i], thresh[i] = roc_curve(testing_labels, pred_prob_labels_roc[:, i], pos_label=i)

# plotting
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Dec_55 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='graminoid_82 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='black', label='mix_conifer_54 vs Rest')
plt.plot(fpr[3], tpr[3], linestyle='--',color='cyan', label='Sb_186 vs Rest')
plt.plot(fpr[4], tpr[4], linestyle='--',color='red', label='Pj_52 vs Rest')
plt.plot(fpr[5], tpr[5], linestyle='--',color='brown', label='Sb_51 vs Rest')
plt.plot(fpr[6], tpr[6], linestyle='--',color='magenta', label='bog_86 vs Rest')
plt.plot(fpr[7], tpr[7], linestyle='--',color='maroon', label='lichen_87 vs Rest')
plt.plot(fpr[8], tpr[8], linestyle='--',color='yellow', label='burn_200 vs Rest')
plt.plot(fpr[9], tpr[9], linestyle='--',color='blue', label='water_80 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);
#
# ########################

In [ ]:
print("our reducted OOB prediction of accuracy for clasifier_imp is {oob}% " .format(oob=classifier_imp.oob_score_*100))

print('initial_Correct red Prediction (%): ', accuracy_score(testing_labels,predicted_imp_labels, normalize=True)*100.0)
#print('Correct Prediction (%): ', accuracy_score(testing_labels,predicted_labels, normalize=True)*100.0)
#*********************************************************************************************************
df=pd.DataFrame()
df['truth'] =testing_labels
df['predict']=predicted_imp_labels

#crosstabulate predictions
print(pd.crosstab(df['truth'],df['predict'],margins=True))

# #confusion metrix***************************
cm=confusion_matrix(testing_labels,predicted_imp_labels )

plt.figure(figsize=(10,7))
sn.heatmap(cm, annot=True)
plt.xlabel('predict')
plt.ylabel('truth')
plt.show()
plt.savefig('cm.png')


In [ ]:

#runing diferent number of trees to see accuracy effect
trees = range(180)
accuracy = np.zeros(180)
for i in range(len(trees)):
    classifier_imp = RandomForestClassifier( n_estimators=i + 1)
    #classifier1=classifier1.fit(training_samples, training_labels)  #
    classifier_imp.fit(imp_training_samples, training_labels)
    #prediction = classifier_imp.predict(imp_testing_samples)
    predicted_labels = classifier_imp.predict(imp_testing_samples)  #==predicted_imp_labels = classifier_imp.predict(imp_testing_samples )
    accuracy[i]= accuracy_score(testing_labels,predicted_labels)

plt.cla()
plt.plot(trees,accuracy)
plt.savefig('efectAccur.png')
plt.show()

### 13. APPLY THE BEST MODEL TO ENTIRE IMAGE

In [ ]:
# predict for entire image**************************
# we used trained object to clasify all the input image.. Predict function expects a list of poxels not
# an matrix. #now, we use trained object ("classifier") to classify entire image
# #
n_samples = rows*cols

flat_pixels = imp_bands_data.reshape((n_samples, needed_bands)) #3D image is reshaped into 2D
print("fp_shp:", flat_pixels.shape)
pred_img = classifier_imp2.predict(flat_pixels)
# #print("imp res:"),pred_img
#classifier_imp2
classification = pred_img.reshape((rows, cols))
write_geotiff(red_output,classification,geo_transform, proj)
fin_pred_labels = classification[is_test]
#*********************************************************************************************
print("confussion matrix:\n%s" % metrics.confusion_matrix(testing_labels, fin_pred_labels))
target_names = ['class %s' % s for s in v_classes]

print("classification report:\n%s" % metrics.classification_report(testing_labels, fin_pred_labels, target_names=target_names))
print("classification accuracy: %f" % metrics.accuracy_score(testing_labels, fin_pred_labels))
#

#visualize data
#import matplotlib
import matplotlib.pyplot as plt
f = plt.figure()
f.add_subplot(1,2,2)
r = bands_data[:,:,5]
g = bands_data[:,:,4]
b = bands_data[:,:,3]
rgb = np.dstack([r,g,b])
f.add_subplot(1, 2, 1)
plt.imshow(rgb)
f.add_subplot(1, 2, 2)
plt.imshow(classification)
plt.savefig('Alosclassification.png')

